In [ ]:
import os
os.environ["CC"] = "/usr/bin/gcc-11"
os.environ["CXX"] = "/usr/bin/g++-11"
import struct
import numpy as np
from tqdm import tqdm

from pathlib import Path
import imageio
import os
import sys
sys.path.append(str(Path(os.path.abspath('')).parent))
print(str(Path(os.path.abspath('')).parent))
import torch
from gaussian_renderer import GaussianModel, splinerender, render
from scene import Scene
from scene.cameras import Camera, MiniCam
from torch import nn
import matplotlib.pyplot as plt
import imageio
from pyquaternion import Quaternion
from scene.dataset_readers import ProjectionType

dataset = "nyc"
path = "train5"
eval_path = "/home/gaussian-splatting-merge/eval.znf"
output_path = Path(f"~/Videos/popping_paths/{path}/ours")

output_path = Path(f"~/Videos/nyc1/ours/")
output_path.mkdir(parents=True, exist_ok=True)
f = open(f"/home/Videos/nyc1.path", "rb")
data = f.read()
N = int.from_bytes(data[:4])
camera_size = 11

cameras = np.array(struct.unpack(f'>{N*camera_size}f', data[4:])).reshape(N, -1)
full_data = struct.unpack(f'>i{N*camera_size}f', data)
N = full_data[0]
cameras = np.array(full_data[1:]).reshape(N, -1)
print(N)

In [ ]:
import shlex
from argparse import ArgumentParser, Namespace
from arguments import ModelParams, PipelineParams, OptimizationParams

def get_combined_args(args_cmdline):
    cfgfile_string = "Namespace()"

    try:
        cfgfilepath = os.path.join(args_cmdline.model_path, "cfg_args")
        print("Looking for config file in", cfgfilepath)
        with open(cfgfilepath) as cfg_file:
            print("Config file found: {}".format(cfgfilepath))
            cfgfile_string = cfg_file.read()
    except TypeError:
        print("Config file not found at")
        pass
    args_cfgfile = eval(cfgfile_string)

    merged_dict = vars(args_cfgfile).copy()
    for k, v in vars(args_cmdline).items():
        if v != None:
            merged_dict[k] = v
    return Namespace(**merged_dict)


parser = ArgumentParser(description="Testing script parameters")
model = ModelParams(parser, sentinel=True)
pipeline = PipelineParams(parser)
args = parser.parse_args(shlex.split(f"-m {Path(eval_path) / dataset} --images images_2 -r 1"))
print(args.model_path)
args = get_combined_args(args)
model = model.extract(args)
model.source_path = str(Path("/data/nerf_datasets/zipnerf_ud") / dataset)

model.max_opacity = 0.99

pipeline = pipeline.extract(args)

In [ ]:
gaussians = GaussianModel(model.sh_degree, model.max_opacity)
scene = Scene(model, gaussians, load_iteration=-1, shuffle=False)

In [11]:
refcam = scene.getTrainCameras()[0]

In [12]:
from gaussian_renderer.fast_renderer import FastRenderer

renderer = FastRenderer(refcam, gaussians, False)


In [ ]:
background = torch.tensor([0, 0, 0], dtype=torch.float32, device="cuda")
width = refcam.image_width
height = refcam.image_height
image = torch.ones((3, height, width), dtype=float)
for i in tqdm(range(N)):
    T = cameras[i, :3]
    # xyzw
    quat = cameras[i, 3:7]
    R = Quaternion(x=quat[0], y=quat[1], z=quat[2], w=quat[3]).transformation_matrix
    R[:3, 3] = T
    transf = np.linalg.inv(R).T
    transf[:, 1] = -transf[:, 1]
    transf[:, 2] = -transf[:, 2]

    fovy = cameras[i, -4]
    fovx = cameras[i, -3]
    fovy = refcam.FoVy
    fovx = refcam.FoVx
    znear = cameras[i, -2]
    zfar = cameras[i, -1]
    world_view_transform = torch.as_tensor(transf).float()
    full_proj_transform = torch.as_tensor(transf).float()
    view = MiniCam(width, height, fovy, fovx, znear, zfar, world_view_transform, full_proj_transform)
    view.model = ProjectionType.PERSPECTIVE
    with torch.no_grad():
        renderer.set_camera(view)
        rendering = renderer.render(view, pipeline, background)
        byte_rendering = (rendering.permute(1, 2, 0).cpu().numpy()*255).clip(min=0, max=255).astype(np.uint8)
    full_output_path = output_path / f"{i:06d}.png"
    imageio.imwrite(str(full_output_path), byte_rendering)